In [1]:
import openmm as mm
from openmm import app, unit
from openmm.app import PDBFile

In [2]:
with open('system_with_ligand_only.xml', 'r') as f:
    system_1 = mm.XmlSerializer.deserialize(f.read())
pdb = PDBFile('ligand_in_solvent.pdb')
system_1.addForce(mm.MonteCarloBarostat(1*unit.atmosphere, 300*unit.kelvin))

5

In [4]:
#NPT simulations for 10 ns
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
simulation = app.Simulation(pdb.topology, system_1, integrator)
#
simulation.context.setPositions(pdb.positions)
integrator.setRandomNumberSeed(42)
print("Simulation is running on:", simulation.context.getPlatform().getName())
simulation.minimizeEnergy(maxIterations=1000)
# save the minimized structure for each replicate as starting-point geometry (almost identical actually...)
with open(f'min_complex.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(), f)
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
# NPT for 10 ns
simulation.reporters.append(app.DCDReporter(f'trajectory.dcd', 5000))
simulation.reporters.append(
                            app.StateDataReporter(
                                                    f'energy_data.csv',
                                                    5000,
                                                    step=True,
                                                    time=True, 
                                                    potentialEnergy=True,
                                                    kineticEnergy=True,
                                                    totalEnergy=True,
                                                    temperature=True,
                                                    volume=True,
                                                    density=True,
                                                    speed=True,
                                                    )
                            )
simulation.step(5000000)
simulation.reporters.clear()

Simulation is running on: CUDA
